# **PV and Battery Sizing Problem**

## Sets

$\mathcal{K} = \{1, 2, \dots, K\}$: Set of time intervals.

$\mathcal{N} = \{1, 2, \dots, N\}$: Set of users (optional, for community-based problem).

---

## Indices

$k \in \mathcal{K}$: Time index.

$i \in \mathcal{N}$: User index (optional, for community-based problem).

---

## Data
### 1. Energy Parameters:
$Y_k, \forall k \in \mathcal{K}$: Solar energy generation per $m^2$ of PV panel at time $k$ (kWh/m²).

$X_k, \forall k \in \mathcal{K}$: Energy demand at time $k$ (kWh).

### 2. Battery Parameters:
$\gamma$: Energy retention rate of the battery ( $\gamma \in (0, 1]$).

$R$: Maximum charge/discharge rate as a fraction of battery capacity ($R \in (0, 1]$).

$\bar{C}$: Maximum capacity of the battery (kWh).

$\alpha$: Minimum state-of-charge (SoC) as a fraction of battery capacity ($\alpha \in [0, 1]$).

$C_0$: Initial state-of-charge (SoC) of the battery (kWh).

### 3. Physical Limits:
$a_{\text{max}}$: Maximum allowable PV panel area (m²).



### 4. Economic Parameters:
$5_{PV}$: Cost of installing 1 $m^2$ of PV panel ($\$/m^2$).

$5_B$: Cost of installing 1 kWh of battery capacity ($\$/kWh$).

$5_R$: Cost (or profit) per unit of reverse power flow ($\$/kWh$).

$5_G$: Cost per unit of energy supplied by fuel cells ($\$/kWh$).

---

## Decision Variables
### 1. PV Panel Area:
$a$: Area of PV panels installed (m²).

### 2. Battery State of Charge:
$C_k, \forall k \in \mathcal{K}$: State-of-charge (SoC) of the battery at time $k$ (kWh).

$\phi_k^+, \forall k \in \mathcal{K}$: Reverse power flow (excess energy injected into the grid) at time $k$ (kWh).

$\phi_k^-, \forall k \in \mathcal{K}$: Energy supplied by fuel cells at time $k$ (kWh).

### 3. Battery Capacity:
$\bar{C}$: Total battery capacity (kWh).

---

# **PV and Battery Sizing Problem**

## Linear Program

$$
\underset{\{a, \bar{C}, C_k, \phi_k^+, \phi_k^-, \forall k \in \mathcal{K}\}}{\text{minimize}} \quad
5_{PV} a + 5_B \bar{C} + \sum_{k \in \mathcal{K}} \left( 5_R \phi_k^+ + 5_G \phi_k^- \right)
$$

### Subject to:

1. **Battery Dynamics**:
   $$
   C_k + \phi_k^+ - \phi_k^- = \gamma C_{k-1} + a Y_{k-1} - X_{k-1}, \quad \forall k \in \mathcal{K}
   $$

2. **Battery Capacity Limits**:
   $$
   \alpha \bar{C} \leq C_k \leq \bar{C}, \quad \forall k \in \mathcal{K}
   $$

3. **Charging/Discharging Rate**:
   $$
   |C_k - C_{k-1}| \leq R \bar{C}, \quad \forall k \in \mathcal{K}
   $$

4. **Non-Negativity**:
   $$
   \phi_k^+ \geq 0, \quad \phi_k^- \geq 0, \quad \forall k \in \mathcal{K}
   $$

5. **PV Panel Area Limit**:
   $$
   0 \leq a \leq a_{\text{max}}
   $$

6. **Initial Battery State**:
   $$
   C_0 = \hat{C}, \quad \hat{C} \in [\alpha \bar{C}, \bar{C}]
   $$

7. **Net-Zero Energy Constraint** (Optional):
   $$
   a \sum_{k \in \mathcal{K}} Y_k \geq \sum_{k \in \mathcal{K}} X_k
   $$

---




In [ ]:
!pip install cplex
!pip install docplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.5/645.5 kB 17.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docplex: filename=docplex-2.29.241-py3-none-any.whl size=685423 sha256=f5002b23f16c56dcdc0602b1d9581a92a34b427a383060132f454a91c218c9bc
  Stored in directory: /root/.cache/pip/wheels/2b/15/6e/eba2382ab6bc3f35194a62d3b007916ececf6f07139cac6795
Successfully built docplex


In [ ]:
from docplex.mp.model import Model

# Create a CPLEX model
model = Model(name="PV_and_Battery_Sizing")

 High Solar Generation, Moderate Energy Demand

In [ ]:
# Input Data
K = 48  # Number of time intervals (e.g., 24 hours in 30-minute steps)
# Solar energy generation (kWh/m²)
Y = [0.0] * 10 + [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5,3.0, 3.5, 4.0, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 1.5,1.0] + [0.0] * 10

# Energy demand (kWh)
X = [3.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 6.5, 6.0, 5.5] + \
    [5.0, 4.5, 4.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5, 7.0, 6.5] + \
    [6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 6.5] + \
    [7.0, 6.5, 6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5]



gamma = 0.95  # Battery retention rate
R = 0.5  # Maximum charge/discharge rate (fraction of capacity)
alpha = 0.2  # Minimum SoC as a fraction of battery capacity
a_max = 200  # Maximum allowable PV panel area (m²)
PV_cost = 1000  # Cost per m² of PV panels (¥/m²)
Battery_cost = 500  # Cost per kWh of battery capacity (¥/kWh)
Reverse_power_cost = 200  # Penalty for reverse power flow (¥/kWh)
Fuel_cost = 50  # Cost of fuel cell energy (¥/kWh)


Low Solar Generation, High Evening Demand

In [ ]:
K = 48  # Number of time intervals (e.g., 24 hours in 30-minute steps)

# Solar energy generation (kWh/m²)
# 0 generation at night, peaks during midday
Y = [0.0] * 10 + [3.0, 3.5, 4.0, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 1.0,0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5] + [0.0] * 10

# Energy demand (kWh)
# Higher demand during morning, evening, and moderate during the day
X = [5.0, 6.5, 4.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 5.0, 4.0, 3.5] + \
    [3.0, 3.5, 4.0, 4.5, 5.0, 4.5, 3.0, 2.5, 2.0, 1.5, 1.0, 2.0] + \
    [3.5, 4.0, 8.0, 10.0, 7.0, 8.0, 7.5, 8.5, 10.5, 8.0, 9.5, 8.0] + \
    [7.5, 5.0, 4.5, 4.0, 3.5, 1.0, 0.5, 1.0,1.5,0.5,1.0,2.0,1.5]

gamma = 0.95  # Battery retention rate
R = 0.5  # Maximum charge/discharge rate (fraction of capacity)
alpha = 0.2  # Minimum SoC as a fraction of battery capacity
a_max = 200  # Maximum allowable PV panel area (m²)
PV_cost = 1000  # Cost per m² of PV panels (¥/m²)
Battery_cost = 500  # Cost per kWh of battery capacity (¥/kWh)
Reverse_power_cost = 200  # Penalty for reverse power flow (¥/kWh)
Fuel_cost = 50  # Cost of fuel cell energy (¥/kWh)


 Balanced Solar Generation, Morning and Evening Peaks in Demand

In [ ]:
K = 48  # Number of time intervals (e.g., 24 hours in 30-minute steps)

# Solar energy generation (kWh/m²)
# 0 generation at night, peaks during midday
Y = [0.0] * 10 + [3.0, 3.5, 4.0, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5, 1.0,0.5, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 5.5, 5.0, 4.5, 4.0, 3.5, 3.0, 2.5, 2.0, 1.5, 1.0, 0.5] + [0.0] * 10

# Energy demand (kWh)
# Higher demand during morning, evening, and moderate during the day
X = [5.0, 6.5, 4.0, 3.5, 4.0, 4.5, 5.0, 5.5, 6.0, 5.0, 4.0, 3.5] + \
    [3.0, 3.5, 4.0, 4.5, 5.0, 4.5, 3.0, 2.5, 2.0, 1.5, 1.0, 2.0] + \
    [3.5, 4.0, 8.0, 10.0, 7.0, 8.0, 7.5, 8.5, 10.5, 8.0, 9.5, 8.0] + \
    [7.5, 5.0, 4.5, 4.0, 3.5, 1.0, 0.5, 1.0,1.5,0.5,1.0,2.0,1.5]

gamma = 0.95  # Battery retention rate
R = 0.5  # Maximum charge/discharge rate (fraction of capacity)
alpha = 0.2  # Minimum SoC as a fraction of battery capacity
a_max = 200  # Maximum allowable PV panel area (m²)
PV_cost = 1000  # Cost per m² of PV panels (¥/m²)
Battery_cost = 500  # Cost per kWh of battery capacity (¥/kWh)
Reverse_power_cost = 200  # Penalty for reverse power flow (¥/kWh)
Fuel_cost = 50  # Cost of fuel cell energy (¥/kWh)


In [ ]:
# Decision Variables
a = model.continuous_var(name="a", lb=0, ub=a_max)  # PV panel area (m²)
C = model.continuous_var_list(K, name="C", lb=0)  # State-of-charge (SoC) at each time (kWh)
phi_plus = model.continuous_var_list(K, name="phi_plus", lb=0)  # Reverse power flow (kWh)
phi_minus = model.continuous_var_list(K, name="phi_minus", lb=0)  # Fuel cell energy (kWh)
C_max = model.continuous_var(name="C_max", lb=0)  # Total battery capacity (kWh)


In [ ]:

# Objective Function
model.minimize(PV_cost * a + Battery_cost * C_max +
               model.sum(Reverse_power_cost * phi_plus[k] + Fuel_cost * phi_minus[k] for k in range(K)))

In [ ]:
# Constraints

In [ ]:
# Battery dynamics
for k in range(K):
    if k == 0:
        model.add_constraint(C[k] + phi_plus[k] - phi_minus[k] == a * Y[k] - X[k], f"Battery_dynamics_{k}")
    else:
        model.add_constraint(C[k] + phi_plus[k] - phi_minus[k] == gamma * C[k - 1] + a * Y[k] - X[k], f"Battery_dynamics_{k}")

In [ ]:
# Battery capacity limits
for k in range(K):
    model.add_constraint(C[k] >= alpha * C_max, f"Min_SoC_{k}")
    model.add_constraint(C[k] <= C_max, f"Max_SoC_{k}")

In [ ]:
# Charging/discharging rate
for k in range(1, K):
    model.add_constraint(C[k] - C[k - 1] <= R * C_max, f"Charging_rate_{k}")
    model.add_constraint(C[k - 1] - C[k] <= R * C_max, f"Discharging_rate_{k}")

In [ ]:
# Non-Negativity constraints
for k in range(K):
    model.add_constraint(phi_plus[k] >= 0, f"Non_Neg_phi_plus_{k}")
    model.add_constraint(phi_minus[k] >= 0, f"Non_Neg_phi_minus_{k}")

In [ ]:
# PV panel area limit
model.add_constraint(a >= 0, "Non_Neg_a")
model.add_constraint(a <= a_max, "Max_a")

docplex.mp.LinearConstraint[Max_a](a,LE,200)

In [ ]:
# Net-zero energy constraint (optional)
model.add_constraint(a * sum(Y) >= sum(X), "Net_Zero_Energy")

docplex.mp.LinearConstraint[Net_Zero_Energy](76.500a,GE,219.0)

In [ ]:

solution = model.solve(log_output=True)

Version identifier: 22.1.2.0 | 2024-12-10 | f4cec290b
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
LP Presolve eliminated 100 rows and 2 columns.
Reduced LP has 237 rows, 144 columns, and 690 nonzeros.
Presolve time = 0.01 sec. (0.25 ticks)
Initializing dual steep norms . . .

Iteration log . . .
Iteration:     1   Dual objective     =          3112.745098
Iteration:    62   Dual objective     =         25295.687745


In [ ]:

if solution:
    print("Objective Value:", solution.objective_value)
    print("PV Panel Area (m²):", solution[a])
    print("Battery Capacity (kWh):", solution[C_max])
    for k in range(K):
        print(f"Time {k}: SoC = {solution[C[k]]:.2f}, Reverse Power = {solution[phi_plus[k]]:.2f}, Fuel Cell = {solution[phi_minus[k]]:.2f}")
else:
    print("No solution found.")


Objective Value: 25861.549325673535
PV Panel Area (m²): 2.8627450980392157
Battery Capacity (kWh): 6.306227090642693
Time 0: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 6.26
Time 1: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 6.56
Time 2: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 4.06
Time 3: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 3.56
Time 4: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 4.06
Time 5: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 4.56
Time 6: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 5.06
Time 7: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 5.56
Time 8: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 6.06
Time 9: SoC = 1.26, Reverse Power = 0.00, Fuel Cell = 5.06
Time 10: SoC = 4.41, Reverse Power = 1.37, Fuel Cell = 0.00
Time 11: SoC = 6.31, Reverse Power = 4.41, Fuel Cell = 0.00
Time 12: SoC = 6.31, Reverse Power = 8.14, Fuel Cell = 0.00
Time 13: SoC = 6.31, Reverse Power = 4.77, Fuel Cell = 0.00
Time 14: SoC = 6.31, Reverse Power = 2.84, Fuel Cell 